In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests
import csv
# from selenium import webdriver
# from selenium.webdriver.common.by import By 

### Data extraction 

In [8]:
url = 'https://storage.googleapis.com/learning-datasets/sarcasm.json'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # print(data)
else:
      print(f"Failed to retrieve file: {response.status_code}")

In [10]:
data[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [13]:
newsdata = []
for item in data:
    newsdata.append({'headline':item['headline'],'label':item['is_sarcastic']})


with open("data.csv", mode = "w" , newline="", encoding = "utf-8") as file:
    filenames = ['headline','label']
    
    writer = csv.DictWriter(file,fieldnames = filenames)
    writer.writeheader()

    writer.writerows(newsdata)